In [67]:
import gmpy2
import numpy as np
np.set_printoptions(formatter={'int':hex})

def tobin(a, bits=8):
    b = []
    for i in range(bits):
        b.append([(a >> i) & 0x01])
    return np.array(b)

def tohex(b):
    a = 0
    for i in range(len(b)):
        a = a ^ b[i][0] << i
    return a

def gfMatMul(a, b):
    row_a = len(a)
    col_a = len(a[0])
    row_b = len(b)
    col_b = len(b[0])
    if row_a != col_b:
        print("error")
        return -1
    o = []
    for k in range(row_b):
        c = []
        for j in range(col_a):
            t = 0
            for i in range(row_a):
                t ^= a[i][j]& b[k][i]
            c.append(t)
        o.append(c)
    return np.array(o)    

def Affine_Transform (d):
    C = np.array([[1], [1], [0], [0], [0], [1], [1], [0]])
    M = np.array([[1, 0, 0, 0, 1, 1, 1, 1],
                  [1, 1, 0, 0, 0, 1, 1, 1],
                  [1, 1, 1, 0, 0, 0, 1, 1],
                  [1, 1, 1, 1, 0, 0, 0, 1],
                  [1, 1, 1, 1, 1, 0, 0, 0],
                  [0, 1, 1, 1, 1, 1, 0, 0],
                  [0, 0, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 1, 1, 1, 1, 1]])
    return gfMatMul(d, M) ^ C

def Inv_Affine_Transform (d):
    C = np.array([[1], [0], [1], [0], [0], [0], [0], [0]])
    M = np.array([[0, 0, 1, 0, 0, 1, 0, 1],
                  [1, 0, 0, 1, 0, 0, 1, 0],
                  [0, 1, 0, 0, 1, 0, 0, 1],
                  [1, 0, 1, 0, 0, 1, 0, 0],
                  [0, 1, 0, 1, 0, 0, 1, 0],
                  [0, 0, 1, 0, 1, 0, 0, 1],
                  [1, 0, 0, 1, 0, 1, 0, 0],
                  [0, 1, 0, 0, 1, 0, 1, 0]])
    return gfMatMul(d, M) ^ C

def GF8to4 (d):
    M = np.array([[1, 0, 1, 1, 1, 0, 1, 1],
                  [0, 1, 0, 1, 0, 0, 0, 0],
                  [0, 1, 0, 0, 1, 0, 1, 0],
                  [0, 1, 1, 0, 0, 0, 1, 1],
                  [0, 0, 0, 0, 1, 1, 1, 0],
                  [0, 1, 0, 0, 1, 0, 1, 1],
                  [0, 0, 1, 1, 0, 1, 0, 1],
                  [0, 0, 0, 0, 0, 1, 0, 1]])
    o = gfMatMul(d, M)
    #print(o)
    return np.array([o[0:4], o[4:8]])

def GF4to8 (d):
    #print(d)
    a = np.concatenate((d[0],d[1]))
    #print(a)
    M = np.array([[1, 0, 0, 0, 1, 0, 1, 0],
                  [0, 0, 0, 0, 1, 1, 0, 1],
                  [0, 1, 0, 0, 1, 1, 1, 0],
                  [0, 1, 0, 0, 1, 1, 0, 1],
                  [0, 1, 0, 1, 1, 0, 1, 0],
                  [0, 0, 1, 0, 0, 1, 0, 1],
                  [0, 1, 1, 1, 0, 1, 1, 1],
                  [0, 0, 1, 0, 0, 1, 0, 0]])
    o = gfMatMul(a, M)
    #print(o)
    return(o)

def GF4to2 (d):
    M = np.array([[1, 0, 0, 0],
                  [0, 0, 1, 1],
                  [0, 1, 1, 1],
                  [0, 0, 0, 1]])
    o = gfMatMul(d, M)
    return [o[0:2], o[2:4]]

def GF2to4 (d):
    a = np.concatenate((d[0],d[1]))
    M = np.array([[1, 0, 0, 0],
                  [0, 1, 1, 0],
                  [0, 1, 0, 1],
                  [0, 0, 0, 1]])
    o = gfMatMul(a, M)
    return o


def GF2Multiply (c, d):
    a = (tohex(c)<<2 ^ tohex(d))
    if a == 0x0: o = 0x0
    elif a == 0x1: o = 0x0
    elif a == 0x2: o = 0x0
    elif a == 0x3: o = 0x0
    elif a == 0x4: o = 0x0
    elif a == 0x5: o = 0x1
    elif a == 0x6: o = 0x2
    elif a == 0x7: o = 0x3
    elif a == 0x8: o = 0x0
    elif a == 0x9: o = 0x2
    elif a == 0xa: o = 0x3
    elif a == 0xb: o = 0x1
    elif a == 0xc: o = 0x0
    elif a == 0xd: o = 0x3
    elif a == 0xe: o = 0x1
    elif a == 0xf: o = 0x2
    return tobin(o, bits=2)

def GF2Multiply10 (d):
    a = tohex(d)
    if a == 0x0: o = 0x0
    elif a == 0x1: o = 0x2
    elif a == 0x2: o = 0x3
    elif a == 0x3: o = 0x1
    return tobin(o, bits=2)

def GF4Squar (d):
    a = tohex(d)
    if a == 0x0: o = 0x0
    elif a == 0x1: o = 0x1
    elif a == 0x2: o = 0x4
    elif a == 0x3: o = 0x5
    elif a == 0x4: o = 0x3
    elif a == 0x5: o = 0x2
    elif a == 0x6: o = 0x7
    elif a == 0x7: o = 0x6
    elif a == 0x8: o = 0xc
    elif a == 0x9: o = 0xd
    elif a == 0xa: o = 0x8
    elif a == 0xb: o = 0x9
    elif a == 0xc: o = 0xf
    elif a == 0xd: o = 0xe
    elif a == 0xe: o = 0xb
    elif a == 0xf: o = 0xa
    return tobin(o, bits=4)

def GF4SquarB (d):
    a = tohex(d)
    if a == 0x0: o = 0x0
    elif a == 0x1: o = 0x9
    elif a == 0x2: o = 0x2
    elif a == 0x3: o = 0xb
    elif a == 0x4: o = 0x8
    elif a == 0x5: o = 0x1
    elif a == 0x6: o = 0xa
    elif a == 0x7: o = 0x3
    elif a == 0x8: o = 0x6
    elif a == 0x9: o = 0xf
    elif a == 0xa: o = 0x4
    elif a == 0xb: o = 0xd
    elif a == 0xc: o = 0xe
    elif a == 0xd: o = 0x7
    elif a == 0xe: o = 0xc
    elif a == 0xf: o = 0x5
    return tobin(o, bits=4)

def GF4inv (d):
    a = tohex(d)
    if a == 0x0: o = 0x0
    elif a == 0x1: o = 0x1
    elif a == 0x2: o = 0x9
    elif a == 0x3: o = 0xe
    elif a == 0x4: o = 0xd
    elif a == 0x5: o = 0xb
    elif a == 0x6: o = 0x7
    elif a == 0x7: o = 0x6
    elif a == 0x8: o = 0xf
    elif a == 0x9: o = 0x2
    elif a == 0xa: o = 0xc
    elif a == 0xb: o = 0x5
    elif a == 0xc: o = 0xa
    elif a == 0xd: o = 0x4
    elif a == 0xe: o = 0x3
    elif a == 0xf: o = 0x8
    return tobin(o, bits=4)

def GF4Multiply (a, b):
    c = GF4to2(a)
    d = GF4to2(b)
    e = GF2Multiply(c[0], d[0])
    f = GF2Multiply(c[0] ^ c[1], d[0] ^ d[1])
    g = GF2Multiply(c[1], d[1])
    h = GF2Multiply10(g)
    o0 = h ^ e
    o1 = f ^ e
    return GF2to4 (np.array([o0, o1]))

def GF4Affine_Transform (d):
    e = np.concatenate((d[0], d[1]))
    C = np.array([[0], [1], [0], [0], [0], [0], [1], [1]])
    M = np.array([[0, 0, 1, 0, 1, 0, 0, 0],
                  [0, 1, 0, 1, 0, 0, 0, 1],
                  [0, 0, 1, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1],
                  [1, 0, 1, 0, 0, 1, 0, 0],
                  [0, 1, 0, 0, 0, 1, 0, 1],
                  [0, 0, 1, 0, 1, 0, 1, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0]])
    f = gfMatMul(e, M) ^ C
    return np.array([f[0:4], f[4:8]])

def GF4Inv_Affine_Transform (d):
    e = np.concatenate((d[0], d[1]))
    C = np.array([[0], [0], [0], [1], [0], [0], [1], [0]])
    M = np.array([[0, 0, 1, 0, 1, 0, 0, 0],
                  [0, 1, 0, 1, 0, 0, 0, 1],
                  [0, 1, 1, 0, 0, 1, 0, 1],
                  [0, 0, 0, 0, 0, 0, 0, 1],
                  [1, 1, 1, 0, 0, 1, 0, 1],
                  [0, 1, 0, 0, 0, 1, 0, 1],
                  [1, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0]])
    f = gfMatMul(e, M) ^ C
    return np.array([f[0:4], f[4:8]])

def Sbox(a):
    #print(a)
    b = GF8to4 (a)
    c0 = GF4Multiply(b[0], b[1])
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    f = GF4to8([e0, e1])
    return Affine_Transform (f)

def InvSbox(a):
    #print(a)
    b = GF8to4(Inv_Affine_Transform(a))    
    #print(b[0], b[1])
    c0 = GF4Multiply(b[0], b[1])
    #print(c0)
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    return GF4to8([e0, e1])

def GF4Sbox(b):
    c0 = GF4Multiply(b[0], b[1])
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    return GF4Affine_Transform([e0, e1])

def GF4InvSbox(a):
    #print(a)
    b = GF4Inv_Affine_Transform(a)    
    #print(b[0], b[1])
    c0 = GF4Multiply(b[0], b[1])
    #print(c0)
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    return [e0, e1]

a = 0x3f
b = tohex(Sbox(tobin(a)))
print(hex(b))
b = tohex(GF4to8(GF4Sbox(GF8to4(tobin(a)))))
print(hex(b))
c = tohex(InvSbox(tobin(b)))
print(hex(c))
c = tohex(GF4to8(GF4InvSbox(GF8to4(tobin(b)))))
print(hex(c))

0x75
0x75
0x3f
0x3f


In [101]:
def GF8Multiply (a, b):
    c = GF8to4(a)
    d = GF8to4(b)
    e = GF4Multiply(c[0], d[0])
    f = GF4Multiply(c[0] ^ c[1], d[0] ^ d[1])
    g = GF4Multiply(c[1], d[1])
    h = GF4Multiply(g, tobin(0x09, bits=4))
    o0 = h ^ e
    o1 = f ^ e
    return GF4to8 (np.array([o0, o1]))

def gf8MatMul(a, b):
    row_a = len(a)
    col_a = len(a[0])
    row_b = len(b)
    col_b = len(b[0])
    if row_a != col_b:
        print("error")
        return -1
    o = []
    for k in range(row_b):
        c = []
        for j in range(col_a):
            t = 0
            for i in range(row_a):
                #print(hex(a[i][j]), hex(b[k][i]), hex(tohex(GF8Multiply(tobin(a[i][j]), tobin(b[k][i])))))
                t ^= tohex(GF8Multiply(tobin(a[i][j]), tobin(b[k][i])))
            c.append(t)
            #print()
        o.append(c)
    return np.array(o) 

def G4tobin(a, bits=8):
    b = []
    for i in range(bits):
        b.append([(a >> i) & 0x01])
    return np.array([b[0:4], b[4:8]])

def G4tohex(b):
    b = np.concatenate((b[0], b[1]))
    a = 0
    for i in range(len(b)):
        a = a ^ b[i][0] << i
    return a

def GF48Multiply (c, d):
    e = GF4Multiply(c[0], d[0])
    f = GF4Multiply(c[0] ^ c[1], d[0] ^ d[1])
    g = GF4Multiply(c[1], d[1])
    h = GF4Multiply(g, tobin(0x09, bits=4))
    o0 = h ^ e
    o1 = f ^ e
    return np.array([o0, o1])

def gf4MatMul(a, b):
    row_a = len(a)
    col_a = len(a[0])
    row_b = len(b)
    col_b = len(b[0])
    if row_a != col_b:
        print("error")
        return -1
    o = []
    for k in range(row_b):
        c = []
        for j in range(col_a):
            t = 0
            for i in range(row_a):
                #print('#', G4tohex(GF48Multiply(G4tobin(a[i][j]), G4tobin(b[k][i]))))
                t ^= G4tohex(GF48Multiply(G4tobin(a[i][j]), G4tobin(b[k][i])))
            c.append(t)
            #print()
        o.append(c)
    return np.array(o) 

def MixColumn(a):
    M = np.array([[2, 3, 1, 1],
                  [1, 2, 3, 1],
                  [1, 1, 2, 3],
                  [3, 1, 1, 2]])
    return gf8MatMul(a, M)

def InvMixColumn(a):    
    M = np.array([[2, 3, 1, 1],
                  [1, 2, 3, 1],
                  [1, 1, 2, 3],
                  [3, 1, 1, 2]])
    M2 = gf8MatMul(a, M)
    M3 = gf8MatMul(M2, M)
    return gf8MatMul(M3, M)
    #M = np.array([[0xe, 0xb, 0xd, 0x9],
    #              [0x9, 0xe, 0xb, 0xd],
    #              [0xd, 0x9, 0xe, 0xb],
    #              [0xb, 0xd, 0x9, 0xe]])
    #return gf8MatMul(a, M)
    
def GF4MixColumn(a):
    M = np.array([[0x2e, 0x2f, 0x01, 0x01],
                  [0x01, 0x2e, 0x2f, 0x01],
                  [0x01, 0x01, 0x2e, 0x2f],
                  [0x2f, 0x01, 0x01, 0x2e]])
    return gf4MatMul(a, M)

def GF4InvMixColumn(a):
    M = np.array([[0x24, 0x6c, 0x0b, 0x42],
                  [0x42, 0x24, 0x6c, 0x0b],
                  [0x0b, 0x42, 0x24, 0x6c],
                  [0x6c, 0x0b, 0x42, 0x24]])
    return gf4MatMul(a, M)

a = np.array([[4],[255],[0],[1]])
#print(InvMixColumn(MixColumn(a)))
print(MixColumn(a))

b = np.array([G4tohex(GF8to4(tobin(i))) for j in i for i in a])
c = GF4MixColumn(b)
d = np.array([tohex(GF4to8(G4tobin(i))) for j in i for i in c])
print(d)

[[0x13]
 [0xe0]
 [0xf8]
 [0xf1]]
[[0x13]
 [0xe0]
 [0xf8]
 [0xf1]]


In [75]:
a = np.array([[4],[20],[33],[255]])
b = []
e = []
print(a)
for i in a:
    b.append([tohex(Sbox(tobin(i[0])))])
print(np.array(b))

c = MixColumn(b)
print(c)
d = InvMixColumn(c)
print(d)
for i in d:
    e.append([tohex(InvSbox(tobin(i[0])))])
print(np.array(e))
    #print(hex(tohex(sbox(tobin(i<<4^j)))), end=" ")

[[0x4]
 [0x14]
 [0x21]
 [0xff]]
[[0xf2]
 [0xfa]
 [0xfd]
 [0x16]]
[[0x1]
 [0x17]
 [0xd3]
 [0x26]]
[[0xf2]
 [0xfa]
 [0xfd]
 [0x16]]
[[0x4]
 [0x14]
 [0x21]
 [0xff]]


In [4]:
def Sbox(a):
    #print(a)
    b = GF8to4 (a)
    c0 = GF4Multiply(b[0], b[1])
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    e2 = np.concatenate((e0, e1))
    print(e2)
    e3 = Affine_Transform (e2)
    print(e2) 
    f = GF4to8([e0, e1])
    return Affine_Transform (f)

def InvSbox(a):
    #print(a)
    b = GF8to4(Inv_Affine_Transform(a))    
    #print(b[0], b[1])
    c0 = GF4Multiply(b[0], b[1])
    #print(c0)
    c1 = GF4Squar(b[0])
    c2 = GF4SquarB(b[1])
    d = GF4inv(c0^ c1^ c2)
    e0 = GF4Multiply(b[0]^ b[1], d)
    e1 = GF4Multiply(b[1], d)
    return GF4to8([e0, e1])

In [103]:
a = np.array([[4],[20],[33],[255]])
b = []
e = []
#print(a)
for i in a:
    b.append([tohex(Sbox(tobin(i[0])))])
#print(np.array(b))

c = MixColumn(b)
#print(c)
d = InvMixColumn(c)
#print(d)
for i in d:
    e.append([tohex(InvSbox(tobin(i[0])))])
#print(np.array(e))
    #print(hex(tohex(sbox(tobin(i<<4^j)))), end=" ")

In [6]:
print(hex(20))

0x14


In [20]:
a = 0x37
print(hex(tohex(Inv_Affine_Transform(Affine_Transform(tobin(a))))))
print(hex(tohex(GF4to8(GF4Inv_Affine_Transform(GF4Affine_Transform(GF8to4(tobin(a))))))))

0x37
0x37


In [12]:
print(tobin(0x37))
b = GF8to4(tobin(0x37))
c = GF4Affine_Transform(b)
print(c)
c0 = np.array([c[0:4],c[4:8]])
print(c0)
d = GF4to8(c0)
print(d)
print(hex(tohex(d)))


[[0x1]
 [0x1]
 [0x1]
 [0x0]
 [0x1]
 [0x1]
 [0x0]
 [0x0]]
[[[0x0]
  [0x1]
  [0x0]
  [0x1]]

 [[0x1]
  [0x0]
  [0x1]
  [0x1]]]
[ array([[[0x0],
        [0x1],
        [0x0],
        [0x1]],

       [[0x1],
        [0x0],
        [0x1],
        [0x1]]])
 array([], shape=(0, 4, 1), dtype=int64)]
error
-1


TypeError: object of type 'int' has no len()

In [37]:
def GF8Multiply (a, b):
    c = GF8to4(a)
    d = GF8to4(b)
    #print(c)
    #print(d)
    e = GF4Multiply(c[0], d[0])
    f = GF4Multiply(c[0] ^ c[1], d[0] ^ d[1])
    g = GF4Multiply(c[1], d[1])
    h = GF4Multiply(g, tobin(0x09, bits=4))
    o0 = h ^ e
    o1 = f ^ e
    s0 = e
    s1 = f ^e ^g
    s2 = g
    print('###')
    print(GF4to8 (np.array([s0, s1])))
    print(GF4to8 (np.array([s2, np.array([[0],[0],[0],[0]])])))
    return GF4to8 (np.array([o0, o1]))

r = GF8Multiply(tobin(0x34), tobin(0x8))
print(r, hex(tohex(r)))

###
[[0x1]
 [0x1]
 [0x1]
 [0x0]
 [0x1]
 [0x0]
 [0x1]
 [0x1]]
[[0x0]
 [0x0]
 [0x1]
 [0x1]
 [0x0]
 [0x1]
 [0x1]
 [0x1]]
[[0x1]
 [0x1]
 [0x0]
 [0x1]
 [0x1]
 [0x1]
 [0x0]
 [0x1]] 0xbb


In [18]:
g4to8 = [[1, 0, 0, 0, 1, 0, 1, 0],
         [0, 0, 0, 0, 1, 1, 0, 1],
         [0, 1, 0, 0, 1, 1, 1, 0],
         [0, 1, 0, 0, 1, 1, 0, 1],
         [0, 1, 0, 1, 1, 0, 1, 0],
         [0, 0, 1, 0, 0, 1, 0, 1],
         [0, 1, 1, 1, 0, 1, 1, 1],
         [0, 0, 1, 0, 0, 1, 0, 0]]
g8to4 = [[1, 0, 1, 1, 1, 0, 1, 1],
         [0, 1, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 1, 0, 1, 0],
         [0, 1, 1, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 1, 1, 1, 0],
         [0, 1, 0, 0, 1, 0, 1, 1],
         [0, 0, 1, 1, 0, 1, 0, 1],
         [0, 0, 0, 0, 0, 1, 0, 1]]
at    = [[1, 0, 0, 0, 1, 1, 1, 1],
         [1, 1, 0, 0, 0, 1, 1, 1],
         [1, 1, 1, 0, 0, 0, 1, 1],
         [1, 1, 1, 1, 0, 0, 0, 1],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 0, 0],
         [0, 0, 1, 1, 1, 1, 1, 0],
         [0, 0, 0, 1, 1, 1, 1, 1]]
ati   = [[0, 0, 1, 0, 0, 1, 0, 1],
         [1, 0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 1, 0, 0, 1],
         [1, 0, 1, 0, 0, 1, 0, 0],
         [0, 1, 0, 1, 0, 0, 1, 0],
         [0, 0, 1, 0, 1, 0, 0, 1],
         [1, 0, 0, 1, 0, 1, 0, 0],
         [0, 1, 0, 0, 1, 0, 1, 0]]
#print (gfMatMul(g4to8, gfMatMul(at, g8to4)))
a = 0x37
print(GF8to4(tobin(a)))
c = gfMatMul(tobin(a), g8to4)
print(c)
#c = gfMatMul(gfMatMul(tobin(a), g8to4), g4to8)

d = gfMatMul(gfMatMul(gfMatMul(gfMatMul(tobin(a), g8to4), g4to8), at), g8to4)^ gfMatMul(tobin(0x63), g8to4)
print(hex(tohex(d)))

d = gfMatMul(c, gfMatMul(g4to8, gfMatMul(at, g8to4)))^ gfMatMul(tobin(0x63), g8to4)
print(hex(tohex(d)))
const = [[1], [0], [1], [0], [0], [0], [0], [0]]
print(gfMatMul(const, g8to4))
print(gfMatMul(g4to8, gfMatMul(at, g8to4)))
print(gfMatMul(g4to8, gfMatMul(ati, g8to4)))

#print(hex(tohex(gfMatMul(d, g4to8))))
#print(hex(tohex(Affine_Transform(tobin(a)))))
#print(gfMatMul(tobin(0x63), g8to4))
#o = (GF8to4(tobin(a)))
#print(GF8to4(tobin(a)))
#print(hex(tohex(GF4Affine_Transform(np.concatenate((o[0], o[1]))))))

[array([[0x1],
       [0x1],
       [0x0],
       [0x0]]), array([[0x0],
       [0x0],
       [0x0],
       [0x1]])]
[[0x1]
 [0x1]
 [0x0]
 [0x0]
 [0x0]
 [0x0]
 [0x0]
 [0x1]]
0xda
0xda
[[0x0]
 [0x0]
 [0x0]
 [0x1]
 [0x0]
 [0x0]
 [0x1]
 [0x0]]
[[0x0 0x0 0x1 0x0 0x1 0x0 0x0 0x0]
 [0x0 0x1 0x0 0x1 0x0 0x0 0x0 0x1]
 [0x0 0x0 0x1 0x0 0x0 0x1 0x0 0x0]
 [0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x1]
 [0x1 0x0 0x1 0x0 0x0 0x1 0x0 0x0]
 [0x0 0x1 0x0 0x0 0x0 0x1 0x0 0x1]
 [0x0 0x0 0x1 0x0 0x1 0x0 0x1 0x0]
 [0x0 0x0 0x0 0x1 0x0 0x0 0x0 0x0]]
[[0x0 0x0 0x1 0x0 0x1 0x0 0x0 0x0]
 [0x0 0x1 0x0 0x1 0x0 0x0 0x0 0x1]
 [0x0 0x1 0x1 0x0 0x0 0x1 0x0 0x1]
 [0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x1]
 [0x1 0x1 0x1 0x0 0x0 0x1 0x0 0x1]
 [0x0 0x1 0x0 0x0 0x0 0x1 0x0 0x1]
 [0x1 0x0 0x0 0x0 0x0 0x0 0x1 0x0]
 [0x0 0x0 0x0 0x1 0x0 0x0 0x0 0x0]]


In [38]:
M = np.array([[2, 3, 1, 1],
              [1, 2, 3, 1],
              [1, 1, 2, 3],
              [3, 1, 1, 2]])
M2 = gf8MatMul(M, M)
M3 = gf8MatMul(M2, M)
print(M3)

[[0xe 0xb 0xd 0x9]
 [0x9 0xe 0xb 0xd]
 [0xd 0x9 0xe 0xb]
 [0xb 0xd 0x9 0xe]]
